In [1]:
from __future__ import print_function, division
import pprint
import os 
import fnmatch
import os.path as osp
import json
import glob as gb
from six import string_types
from nipype.interfaces.base import Bunch

import utils
reload(utils)

<module 'utils' from 'utils.pyc'>

In [2]:
pp = pprint.PrettyPrinter(indent=2)

In [3]:
#- testing
JBBIDS = osp.join(osp.expanduser('~'),'data','bids','ds105')
BIDS2PYPE = osp.join(osp.expanduser('~'),'code','bids2pype','bids2pype')
TESTBIDS = osp.join(BIDS2PYPE, 'tests','data','ds005')

test_case_105a = {
    'base_dir': JBBIDS,
    'model_pattern': "ds-105_level-run_sub*_model.json"
}
test_case_105b = {
    'base_dir': JBBIDS,
    'model_pattern': "ds-105_level-all_model.json"
}
test_case_005 = {
    'base_dir': JBBIDS,
    'model_pattern': "*_model.json"
}
test_case_005r = {
    'base_dir': JBBIDS,
    'model_pattern': "ds-005_type-russ*_model.json"
}

test_case_005t = {
    'base_dir': TESTBIDS,
    'model_pattern': "ds-005_type-russ*_model.json"
}

test_case = test_case_005t

base_dir, model_pattern = test_case['base_dir'], test_case['model_pattern'] 
print(test_case) 

{'model_pattern': 'ds-005_type-russ*_model.json', 'base_dir': '/home/jb/code/bids2pype/bids2pype/tests/data/ds005'}


## Association between data and model 

In [4]:
# take a model such as ds-005_type-russ*_model.json and associate data to this model
# This is a dictionary with keys being the data and value the model to be applied to
# these data
# Returns: the data dictionary and the base dir
assos_model_data = utils.associate_model_data(base_dir, model_pattern, 
                                                  verbose=utils.VERB['none'])

# The data dictionary  
data_dict = assos_model_data['data_dict']

# Take the first datafile : first key of the data_dict
datafile = sorted(data_dict.keys())[0]
print(datafile)
pp.pprint(data_dict[datafile]['Contrasts'].keys())

/home/jb/code/bids2pype/bids2pype/tests/data/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-01_bold.nii.gz
[u'param-loss-neg', u'param-gain', u'task>Baseline']


## Have a look at the model dictionary for a datafile

In [5]:
# get model dictionary for the datafile (the data, ie. the "Y")
model_dict = data_dict[datafile]
print("\nModel for \n{} :".format(datafile))
pp.pprint(model_dict)
# get the contrasts (the model json file contains the description of the contrasts)
print("\nConstrasts")
contrasts = utils._get_nipype_contrasts(model_dict)
pp.pprint(contrasts)


Model for 
/home/jb/code/bids2pype/bids2pype/tests/data/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-01_bold.nii.gz :
{ u'Columns': { u'motion-param': { u'FileSelector': { u'EntitiesKeys': { u'run': u'run',
                                                                         u'subject': u'sub'},
                                                      u'pattern': u'derivatives/mcflirt/par/_runcode_{0[run]:1d}/_subject_id_sub-{0[sub]}/sub-{0[sub]}_task-mixedgamblestask_run-{0[run]}_bold_mcf.nii.gz.par'},
                                   u'Regressors': [u'all', u'deriv1']},
                u'param-gain': { u'Demean': True,
                                 u'HRFModelling': u'Gamma+derivs',
                                 u'Level': u'n/a',
                                 u'ModulationOrder': 1,
                                 u'ModulationVar': u'gain',
                                 u'Variable': u'trial_type',
                                 u'duration': 1.3932149481304148},

## Get Nipype inputs and bunches for this bids

In [6]:
specifymodel_inputs, bunches, data = \
    utils._get_nipype_specify_model_inputs(base_dir, model_pattern, bunch_type='fsl', 
                                                                 verbose=utils.VERB['none'])

sorted_bunch = [b for (d,b) in sorted(zip(data, bunches))]
sorted_data = sorted(data)
        
print('\nSpecifymodel_inputs:')
pp.pprint(specifymodel_inputs)
print('\nsorted_data[:3]:')
pp.pprint(sorted_data[:3]) 

#print(sorted_bunch[0].conditions)
print("\nsorted_bunch[0].conditions[3]:")
print(sorted_bunch[0].conditions[3])
print("\nsorted_bunch[0].amplitudes[3]:")
print(sorted_bunch[0].amplitudes[3])
#print(index_gain)


Specifymodel_inputs:
{ 'high_pass_filter_cutoff': 80, 'input_units': 'secs', 'time_repetition': 2.0}

sorted_data[:3]:
[ '/home/jb/code/bids2pype/bids2pype/tests/data/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-01_bold.nii.gz',
  '/home/jb/code/bids2pype/bids2pype/tests/data/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-02_bold.nii.gz',
  '/home/jb/code/bids2pype/bids2pype/tests/data/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-03_bold.nii.gz']

sorted_bunch[0].conditions[3]:
param-gain

sorted_bunch[0].amplitudes[3]:
[-5.5581395348837219, -7.5581395348837219, -15.558139534883722, 8.4418604651162781, -7.5581395348837219, 0.44186046511627808, 6.4418604651162781, -9.5581395348837219, -3.5581395348837219, 10.441860465116278, -9.5581395348837219, 4.4418604651162781, -11.558139534883722, 2.4418604651162781, -11.558139534883722, 0.44186046511627808, 14.441860465116278, 2.4418604651162781, -5.5581395348837219, 0.44186046511627808, 6.4418604651162781, -7.5581395348837219, -5

### copy a ds tree with only the json and tsv files

In [7]:
source_dir = '/home/jb/data/bids/ds005'
dest_dir = '/home/jb/code/bids2pype/bids2pype/tests/data/ds005'
# utils.create_empty_bids(source_dir, dest_dir, ['*.json','*.tsv', '*.par'])